In [1]:
import pandas
import sklearn.svm
import sklearn.metrics
import sklearn.preprocessing
import sklearn.model_selection

In [2]:
init_df = pandas.read_csv('winequality-red.csv', sep=',')

In [3]:
x = pandas.concat([init_df.iloc[:, :8], init_df.iloc[:, 9:]], axis=1, join='inner')
y = init_df['pH']

In [4]:
mul_dict = {}
mul_dict['fixed acidity'] = 10
mul_dict['volatile acidity'] = 1000
mul_dict['citric acid'] = 100
mul_dict['residual sugar'] = 10
mul_dict['chlorides'] = 1000
mul_dict['free sulfur dioxide'] = 1
mul_dict['total sulfur dioxide'] = 1
mul_dict['density'] = 100000
mul_dict['sulphates'] = 100
mul_dict['pH'] = 100
mul_dict['alcohol'] = 10
mul_dict['quality'] = 1

In [5]:
for i in x.columns:
    x[i] = x[i] * mul_dict.get(i)

In [6]:
x = x.astype('int64')

In [7]:
y = (y * 100).astype('int64')

In [8]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, test_size=0.25)

In [9]:
logistic_train_source, logistic_predict_source, logistic_train_target, logistic_remove_target = sklearn.model_selection.train_test_split(x_train, y_train, test_size=0.33)

In [10]:
logisticRegr = sklearn.linear_model.LogisticRegression()

In [11]:
logisticRegr.fit(logistic_train_source, logistic_train_target)

/home/andreas/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [12]:
y_pred = logisticRegr.predict(logistic_predict_source)

In [13]:
variance = round(abs(sum(logistic_remove_target - y_pred) / len(logistic_remove_target)) / 100, 3)

In [14]:
x_train_partinion_a = pandas.concat([logistic_train_source.iloc[:, :8], logistic_train_target, logistic_train_source.iloc[:, 8:-1]], axis=1)

In [15]:
x_train_partinion_b = pandas.concat([logistic_predict_source.iloc[:, :8], logistic_remove_target, logistic_predict_source.iloc[:, 8:-1]], axis=1)

In [16]:
x_train = pandas.concat([x_train_partinion_a, x_train_partinion_b], axis=0)

In [17]:
y_train = pandas.concat([logistic_train_source['quality'], logistic_predict_source['quality']], axis=0)

In [18]:
for i in x_train.columns:
    x_train[i] = x_train[i] / mul_dict.get(i)

In [19]:
y_test = x_test['quality']

In [20]:
x_test = pandas.concat([x_test.iloc[:, :8], y_test, x_test.iloc[:, 8:-1]], axis=1)

In [21]:
for i in x_test.columns:
    x_test[i] = x_test[i] / mul_dict.get(i)

In [22]:
svm_clf = sklearn.svm.SVC(gamma=0.01, C=1000, kernel='rbf')
svm_clf.fit(x_train, y_train)

SVC(C=1000, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [23]:
y_pred = svm_clf.predict(x_test)

In [24]:
print(sklearn.metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           3       0.00      0.00      0.00         3
           4       0.02      0.08      0.03        13
           5       0.53      0.48      0.50       161
           6       0.49      0.42      0.45       166
           7       0.24      0.11      0.15        53
           8       0.00      0.00      0.00         4

    accuracy                           0.38       400
   macro avg       0.21      0.18      0.19       400
weighted avg       0.45      0.38      0.41       400



/home/andreas/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
